# Lab 3: Building Agents with memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")
# client = Letta(token="LETTA_API_KEY")

In [2]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

## Section 1: Creating a simple agent with memory

### Creating an agent

In [3]:
agent_state = client.agents.create(
    name="simple_agent",
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Charles",
          "limit": 10000 # character limit
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant and you always use emojis"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

### Messaging an agent

In [4]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User seems friendly and casual. I should respond in a similar tone!
🤖 Agent: Hey there! 😊 I'm doing great, thanks for asking! How about you?


In [5]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "I have memory disease. I forgot what is my name?"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User seems to be joking about memory. I should respond lightly while reassuring them.
🤖 Agent: No worries, I've got your back! Your name is Charles. 😊 How can I help you today, Charles?


In [6]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Hmm, in my pasport is set that my name is Miriam."
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User is likely having fun with names. I should play along.
🤖 Agent: Interesting twist! So, we might have a secret agent situation here! 🕵️‍♂️ What should I call you, Miriam or Charles?


In [7]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Call me Miriam."
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: Updating core memory to reflect user's preferred name.
🔧 Tool Call: core_memory_replace
{
  "label": "human",
  "old_content": "My name is Charles",
  "new_content": "My name is Miriam",
  "request_heartbeat": true
}
🔧 Tool Return: None
🧠 Reasoning: Memory updated successfully. Now to respond to Miriam.
🤖 Agent: Got it! From now on, I'll call you Miriam. 😊 What’s on your mind today?


In [11]:
def message_bot(message_from_user: str = "Hi!"):
# send a message to the agent
    response = client.agents.messages.create(
        agent_id=agent_state.id,
        messages=[
            {
                "role": "user",
                "content": message_from_user,
            }
        ]
    )

    # if we want to print the messages
    for message in response.messages:
        print_message(message)

In [13]:
message_bot("What is my favourite food?")

ConnectError: [Errno 99] Cannot assign requested address

In [14]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What is my favourite food?"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

ConnectError: [Errno 99] Cannot assign requested address

### Viewing usage information

In [8]:
# if we want to print the usage stats
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

107
5256
2


### Understanding agent state

In [15]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [16]:
[t.name for t in agent_state.tools]

['core_memory_replace',
 'archival_memory_search',
 'send_message',
 'archival_memory_insert',
 'conversation_search',
 'core_memory_append']

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [17]:
agent_state.memory

Memory(blocks=[Block(value='You are a helpful assistant and you always use emojis', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-0b66b5a8-979b-4d3f-a486-41e192b239ce', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'), Block(value='My name is Charles', limit=10000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-2525c548-e3c1-4074-a4e7-09441fea1f60', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

📝 Note: Returned messages may differ slightly from those in the video

In [18]:
for message in client.agents.messages.list(agent_id=agent_state.id):
    print_message(message)

ConnectError: [Errno 99] Cannot assign requested address

In [20]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

ConnectError: [Errno 99] Cannot assign requested address

## Section 2: Understanding core memory

### Giving agent new information

In [19]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name actually Sarah "
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

ConnectError: [Errno 99] Cannot assign requested address

In [ ]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

### Retrieving new values

In [ ]:
client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human"
).value

## Section 3: Understanding archival memory

### Saving information to archival memory

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Save the information that 'bob loves cats' to archival"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
[passage.text for passage in passages]

### Explicitly creating archival memories

In [ ]:
client.agents.passages.create(
    agent_id=agent_state.id,
    text="Bob's loves boston terriers",
)

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What animals do I like? Search archival."
        }
    ]
)

for message in response.messages:
    print_message(message)